# Dappier

[Dappier](https://dappier.com) 将任何 LLM 或您的 Agentic AI 连接到来自可信来源的实时、已获授权的专有数据，让您的 AI 成为任何领域的专家。我们的专业模型包括实时网络搜索、新闻、体育、金融股市数据、加密货币数据，以及来自高端出版商的独家内容。请访问 [marketplace.dappier.com](https://marketplace.dappier.com) 浏览我们市场中的各类数据模型。

[Dappier](https://dappier.com) 提供丰富、即时可用且与上下文相关的字符串数据，经过优化，可与 LangChain 无缝集成。无论您是在构建会话式 AI、推荐引擎还是智能搜索，Dappier 的 LLM 无关 RAG 模型都能确保您的 AI 能够访问经过验证的最新数据——无需构建和管理自己的检索管道的复杂性。

# Dappier 工具

这将帮助你开始使用 Dappier [工具](https://python.langchain.com/docs/concepts/tools/)。如需了解 DappierRetriever 所有功能和配置的详细文档，请前往[API 参考](https://python.langchain.com/en/latest/tools/langchain_dappier.tools.Dappier.DappierRealTimeSearchTool.html)。

## 概览

DappierRealTimeSearchTool 和 DappierAIRecommendationTool 为 AI 应用提供实时数据和 AI 驱动的洞察力。前者提供对新闻、天气、旅游和金融市场最新信息的访问，而后者则通过 Dappier 的预训练 RAG 模型和自然语言 API 支持的新闻、金融和体育等领域的基于事实的高级内容来增强应用程序。

### 设置

该工具位于 `langchain-dappier` 包中。

In [ ]:
%pip install -qU langchain-dappier

### 凭证

我们还需要设置Dappier API凭证，可以在[Dappier 网站](https://platform.dappier.com/profile/api-keys)上生成凭证。

In [ ]:
import getpass
import os

if not os.environ.get("DAPPIER_API_KEY"):
    os.environ["DAPPIER_API_KEY"] = getpass.getpass("Dappier API key:\n")

如果你想获取来自单个查询的自动化跟踪，你也可以通过取消注释下面这行来设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [18]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

## DappierRealTimeSearchTool

获取实时的谷歌搜索结果，包括最新新闻、天气、旅行和优惠信息，以及来自 polygon.io 的最新财经新闻、股票价格和交易信息，所有这些都由 AI 洞察技术提供支持，让您时刻保持知情。

### 实例化

- ai_model_id: str
    要用于查询的 AI 模型 ID。AI 模型 ID 始终以 "am_" 前缀开头。

    默认为 "am_01j06ytn18ejftedz6dyhz2b15"。

    有多个可用的 AI 模型 ID，可以在以下位置找到：
    https://marketplace.dappier.com/marketplace

In [4]:
from langchain_dappier import DappierRealTimeSearchTool

tool = DappierRealTimeSearchTool(
    # ai_model_id="...",     # overwrite default ai_model_id
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)

### 调用

#### [直接使用参数调用](/docs/concepts/tools)

`DappierRealTimeSearchTool` 接受一个名为 "query" 的参数，该参数应为自然语言查询：

In [5]:
tool.invoke({"query": "What happened at the last wimbledon"})

"At the last Wimbledon in 2024, Carlos Alcaraz won the title by defeating Novak Djokovic. This victory marked Alcaraz's fourth Grand Slam title at just 21 years old! 🎉🏆🎾"

### [使用 ToolCall 调用](/docs/concepts/tools)

我们也可以使用模型生成的 ToolCall 来调用工具，在这种情况下将返回一个 ToolMessage：

In [6]:
# This is usually generated by a model, but we'll create a tool call directly for demo purposes.
model_generated_tool_call = {
    "args": {"query": "euro 2024 host nation"},
    "id": "1",
    "name": "dappier",
    "type": "tool_call",
}
tool_msg = tool.invoke(model_generated_tool_call)

# The content is a JSON string of results
print(tool_msg.content[:400])

Euro 2024 is being hosted by Germany! 🇩🇪 The tournament runs from June 14 to July 14, 2024, featuring 24 teams competing across various cities like Berlin and Munich. It's going to be an exciting summer of football! ⚽️🏆


### 链式调用

我们可以通过先将我们的工具绑定到[工具调用模型](/docs/how_to/tool_calling/)，然后调用它来实现链式调用：

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

In [11]:
# | output: false
# | echo: false

# !pip install -qU langchain langchain-openai
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="gpt-4o", model_provider="openai", temperature=0)

In [12]:
import datetime

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

today = datetime.datetime.today().strftime("%D")
prompt = ChatPromptTemplate(
    [
        ("system", f"You are a helpful assistant. The date today is {today}."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

# specifying tool_choice will force the model to call this tool.
llm_with_tools = llm.bind_tools([tool])

llm_chain = prompt | llm_with_tools


@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = tool.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


tool_chain.invoke("who won the last womens singles wimbledon")

AIMessage(content="Barbora Krejčíková won the women's singles title at Wimbledon 2024, defeating Jasmine Paolini in the final with a score of 6–2, 2–6, 6–4. This victory marked her first Wimbledon singles title and her second major singles title overall! 🎉🏆🎾", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 222, 'total_tokens': 291, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None}, id='run-87a385dd-103b-4344-a3be-2d6fd1dcfdf5-0', usage_metadata={'input_tokens': 222, 'output_tokens': 69, 'total_tokens': 291, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## DappierAIRecommendationTool

使用 Dappier 预训练的 RAG 模型和自然语言 API，为您的 AI 应用注入强大动力，通过涵盖新闻、金融、体育、天气等多个垂直领域的高质量内容提供商，提供事实准确且最新的响应。

### 初始化

- data_model_id: str  
  用于推荐的数据模型 ID。数据模型 ID 始终以 "dm_" 前缀开头。默认值为 "dm_01j0pb465keqmatq9k83dthx34"。  
  有多个数据模型 ID 可供选择，可在 [Dappier marketplace](https://marketplace.dappier.com/marketplace) 找到。  

- similarity_top_k: int  
  根据相似度检索的顶部文档数量。默认值为 "9"。  

- ref: Optional[str]
  应显示 AI 推荐的网站域名。默认为 "None"。  

- num_articles_ref: int
  从指定的参考域名 ("ref") 返回的最小文章数。其余文章将来自 RAG 模型中的其他网站。默认为 "0"。  

- search_algorithm: Literal["most_recent", "semantic", "most_recent_semantic", "trending"]
  用于检索文章的搜索算法。默认为 "most_recent"。

In [15]:
from langchain_dappier import DappierAIRecommendationTool

tool = DappierAIRecommendationTool(
    data_model_id="dm_01j0pb465keqmatq9k83dthx34",
    similarity_top_k=3,
    ref="sportsnaut.com",
    num_articles_ref=2,
    search_algorithm="most_recent",
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)

### 调用

#### [直接使用参数调用](/docs/concepts/tools)

`DappierAIRecommendationTool` 接受一个名为 "query" 的参数，它应该是一个自然语言查询：

In [16]:
tool.invoke({"query": "latest sports news"})

[{'author': 'Matt Weaver',
  'image_url': 'https://images.dappier.com/dm_01j0pb465keqmatq9k83dthx34/Screenshot_20250117_021643_Gallery_.jpg?width=428&height=321',
  'pubdate': 'Fri, 17 Jan 2025 08:04:03 +0000',
  'source_url': 'https://sportsnaut.com/chili-bowl-thursday-bell-column/',
  'summary': "The article highlights the thrilling unpredictability of the Chili Bowl Midget Nationals, focusing on the dramatic shifts in fortune for drivers like Christopher Bell, Tanner Thorson, and Karter Sarff during Thursday's events. Key moments included Sarff's unfortunate pull-off and a last-lap crash that allowed Ryan Bernal to capitalize and improve his standing, showcasing the chaotic nature of the race and the importance of strategy and luck.\n\nAs the competition intensifies leading up to Championship Saturday, Bell faces the challenge of racing from a Last Chance Race, reflecting on the excitement and difficulties of the sport. The article emphasizes the emotional highs and lows experienced

### [使用 ToolCall 调用](/docs/concepts/tools)

我们也可以使用模型生成的 ToolCall 来调用工具，在这种情况下将返回一个 ToolMessage：

In [17]:
# This is usually generated by a model, but we'll create a tool call directly for demo purposes.
model_generated_tool_call = {
    "args": {"query": "top 3 news articles"},
    "id": "1",
    "name": "dappier",
    "type": "tool_call",
}
tool_msg = tool.invoke(model_generated_tool_call)

# The content is a JSON string of results
print(tool_msg.content[:400])

[{"author": "Matt Johnson", "image_url": "https://images.dappier.com/dm_01j0pb465keqmatq9k83dthx34/MLB-New-York-Mets-at-Colorado-Rockies-23948644_.jpg?width=428&height=321", "pubdate": "Fri, 17 Jan 2025 13:31:02 +0000", "source_url": "https://sportsnaut.com/new-york-mets-rumors-vladimir-guerrero-jr-news/", "summary": "The New York Mets are refocusing their strategy after failing to extend a contra


## API 参考

有关 DappierRealTimeSearchTool 所有功能和配置的详细文档，请访问 [API 参考](https://python.langchain.com/api_reference/community/tools/langchain_dappier.tools.dappier.tool.DappierRealTimeSearchTool.html)。